In [2]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

In [3]:
url = "https://www.autocasion.com/coches-ocasion"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [6]:
anuncios_pagina = soup.find_all("article", class_="anuncio")

total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios_pagina)) + (1 if (total_coches/len(anuncios_pagina))%1 else 0)

total_paginas

6541

In [8]:
def links_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?page="
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    lista_paginas = []
    anuncios_coches = []

    for pagina in range(0, total_paginas +1):
        lista_paginas.append(f'{url_pagina}{pagina}')
        
    lista_prueba = lista_paginas[:5]
        
    for pagina_url in lista_prueba:
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("article", class_="anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')   
    
        
    return anuncios_coches

In [22]:
anuncios_coches = links_coches()
anuncios_coches

['https://www.autocasion.com/coches-segunda-mano/tesla-model-x-ocasion/model-x-long-range-awd-ref11307969',
 'https://www.autocasion.com/coches-segunda-mano/land-rover-range-rover-evoque-ocasion/2-0-d-150cc-s-awd-bva-166f7173ae2560-ref14438493',
 'https://www.autocasion.com/coches-segunda-mano/volkswagen-tiguan-ocasion/tiguan-todoterreno-2-0-tdi-110kw-dsg-4wd-sport-150-5p-ref14609369',
 'https://www.autocasion.com/coches-segunda-mano/peugeot-3008-suv-ocasion/3008-1-5bluehdi-gt-line-s-s-eat8-130-ref14604063',
 'https://www.autocasion.com/coches-segunda-mano/volkswagen-golf-ocasion/golf-1-5-tsi-evo-bm-last-edition-96kw-ref14609023',
 'https://www.autocasion.com/coches-segunda-mano/hyundai-tucson-ocasion/tucson-1-6crdi-48v-sle-4x2-ref14610747',
 'https://www.autocasion.com/coches-segunda-mano/bmw-x3-ocasion/x3-xdrive-30e-1-ref14604035',
 'https://www.autocasion.com/coches-segunda-mano/skoda-kodiaq-ocasion/kodiaq-2-0-tsi-style-4x4-dsg-140kw-1-ref13972064',
 'https://www.autocasion.com/coch

In [32]:
url_coche = anuncios_coches[0]
headers = {
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
session = requests.Session()
response = session.get(url_coche, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

In [156]:
provincia = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[-1].text

distintivo_ambiental = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']

marca_modelo = soup.find('h2').text

detalles = soup.find('ul', class_= 'datos-basicos-ficha').find_all('li')
lista_detalles = []

for detalle in detalles:
    lista_detalles.append(detalle.text.strip())
    
    

In [158]:
lista_detalles

['Matriculado: 12/2019',
 '170.000 km',
 'Garantía: 48 meses',
 'Automático',
 'Corriente eléctrica',
 'Todo Terreno',
 '5 asientos',
 '525 cv',
 '5 Puertas',
 'Blanco']